# Cayley-Dickson Construction Applied to Qi Definition

*Version 1*

My original implementation of Gaussian integers included two classes, ``Zi`` and ``Qi``, where, for example, ``Zi(2, -7)`` represents a Gaussian integer, and ``Qi(-2/3, 4/5)`` represents a Gaussian rational.

I'd like to extend this code to include rational-valued quaternions and octonions using the Cayley-Dickson construction.

The [Cayley-Dickson construction](https://en.wikipedia.org/wiki/Cayley%E2%80%93Dickson_construction) is a process by which one can use a recursive definition of conjugation together with a recursive definition of multiplication to use...
* pairs of real numbers ($\mathbb{R}$) to create complex numbers,
* pairs of complex numbers ($\mathbb{C}$) to create quaternions,
* pairs of quaternions ($\mathbb{H}$) to create octonions,
* pairs of octonions ($\mathbb{O}$) to create sedenions ($\mathbb{S}$), and so on.

For more specifics, see my write-up about the Cayley-Dickson construction [at this link](https://abstract-algebra.readthedocs.io/en/latest/55_cayley_dickson.html).

In [1]:
from cayley_dickson_alg import Zi, SetScalarMult
from random import randint

In [2]:
from random import seed

seed(42)  # Generate the same random sequence each time (for testing)

In [3]:
from fractions import Fraction
from functools import wraps

In [4]:
def gaussian_rational(fnc):
    """For use as a property that casts an argument into Gaussian rational."""
    @wraps(fnc)
    def gaussian_rational_wrapper(arg, num):
        qi = to_gaussian_rational(num)
        return fnc(arg, qi)
    return gaussian_rational_wrapper

In [5]:
class Qi:
    """Gaussian Rational Number Class"""

    __MAX_DENOMINATOR = 1_000_000

    def __init__(self, re=None, im=None):

        # CLAUSE AA --------------------------------------------------------
        # re is a str, float, int, or Fraction; and
        # im is a str, float, int, Fraction, or None

        if isinstance(re, (str, float, int, Fraction)):
            if isinstance(re, Fraction):
                self.__re = re
            else:
                self.__re = Fraction(re)
            if im is None:
                self.__im = Fraction(0)
            elif isinstance(im, (str, float, int, Fraction)):
                if isinstance(im, Fraction):
                    self.__im = im
                else:
                    self.__im = Fraction(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE BA --------------------------------------------------------
        # re is a complex; and
        # im is complex, a Qi, or None

        elif isinstance(re, complex):
            if im is None:
                self.__re = Fraction(re.real)
                self.__im = Fraction(re.imag)
            elif isinstance(im, (Qi, complex)):
                self.__re = Qi(re.real, re.imag)
                self.__im = Qi(im.real, im.imag)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE CA --------------------------------------------------------
        # re is a Qi, and im is complex, a Qi, or None

        elif isinstance(re, Qi):
            if im is None:
                self.__re = re.real
                self.__im = re.imag
            elif isinstance(im, (complex, Qi)):
                self.__re = Qi(re)
                self.__im = Qi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE DA --------------------------------------------------------
        # re is a Zi, and im is a Zi, or None

        elif isinstance(re, Zi):
            if im is None:
                if isinstance(re.real, Zi) and isinstance(re.imag, Zi):
                    self.__re = Qi(re.real)
                    self.__im = Qi(re.imag)
                else:
                    self.__re = re.real
                    self.__im = re.imag
            elif isinstance(im, Zi) and im.order() == re.order():
                if isinstance(re.real, Zi) and isinstance(re.imag, Zi):
                    self.__re = Qi(re)
                    self.__im = Qi(im)
                else:
                    self.__re = re.real
                    self.__im = re.imag
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE EA --------------------------------------------------------
        # re is a list or tuple of numbers with length equal to a
        # power of 2, and im is None, or it is a tuple or list
        # similar to the one input for re.
        elif isinstance(re, (tuple, list)):
            z = Zi.from_array(re)
            if im is None:
                self.__re = z.real
                self.__im = z.imag
            elif isinstance(im, (tuple, list)) and len(im) == len(re):
                w = Zi.from_array(im)
                self.__re = z
                self.__im = w
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")

        # CLAUSE FA --------------------------------------------------------
        # Both re and im are None

        elif re is None:
            self.__re = 0
            if im is None:
                self.__im = 0
            else:
                raise Exception(f"If re is None, then im must be None. But im = {im}")
        else:
            raise Exception(f"Unexpected combination of input types: {re} and {im}")

    @classmethod
    def max_denominator(cls, denom=None):
        if denom is None:
            return cls.__MAX_DENOMINATOR
        elif isinstance(denom, int) and denom > 1:
            cls.__MAX_DENOMINATOR = denom
            return cls.__MAX_DENOMINATOR
        else:
            raise ValueError(f"Maximum denominator, {denom}, must be an integer > 1")

    @property
    def real(self) -> Fraction:
        return self.__re

    @property
    def imag(self) -> Fraction:
        return self.__im

    def __repr__(self):
        return f"{self.__class__.__name__}({self.__re}, {self.__im})"

    def __str__(self):
        if self.__im < 0:
            return f"({self.__re}{self.__im}j)"
        else:
            return f"({self.__re}+{self.__im}j)"

    def __neg__(self):
        """Negate this Qi."""
        return Qi(-self.__re, -self.__im)

    def __eq__(self, other):
        """Return True if this Qi equals other."""
        return (self.__re == other.real) and (self.__im == other.imag)

    def __ne__(self, other):
        """Return True if this Qi does NOT equal other."""
        return (self.__re != other.real) or (self.__im != other.imag)

    @gaussian_rational
    def __add__(self, other):
        return Qi(self.__re + other.real, self.__im + other.imag)
    
    def __sub__(self, other):
        return Zi(self.__re - other.real, self.__im - other.imag)

## Generate Unittests

In [9]:
import itertools

opt1 = [('str', '1/2'), ('float', 0.75), ('int', -3), ('Fraction', Fraction(3, 4))]
opt2 = [('str', '1/2'), ('float', 0.75), ('int', -3), ('Fraction', Fraction(3, 4)), ('None', None)]

for x, y in itertools.product(opt1, opt2):
    print(f"\n        # {x[0]} - {y[0]}")
    if y[1] is not None:
        print(f"        self.assertEqual(Qi({repr(x[1])}, {repr(y[1])}), {Qi(x[1], y[1])})")
    else:
        print(f"        self.assertEqual(Qi({repr(x[1])}), {Qi(x[1])})")


        # str - str
        self.assertEqual(Qi('1/2', '1/2'), (1/2+1/2j))

        # str - float
        self.assertEqual(Qi('1/2', 0.75), (1/2+3/4j))

        # str - int
        self.assertEqual(Qi('1/2', -3), (1/2-3j))

        # str - Fraction
        self.assertEqual(Qi('1/2', Fraction(3, 4)), (1/2+3/4j))

        # str - None
        self.assertEqual(Qi('1/2'), (1/2+0j))

        # float - str
        self.assertEqual(Qi(0.75, '1/2'), (3/4+1/2j))

        # float - float
        self.assertEqual(Qi(0.75, 0.75), (3/4+3/4j))

        # float - int
        self.assertEqual(Qi(0.75, -3), (3/4-3j))

        # float - Fraction
        self.assertEqual(Qi(0.75, Fraction(3, 4)), (3/4+3/4j))

        # float - None
        self.assertEqual(Qi(0.75), (3/4+0j))

        # int - str
        self.assertEqual(Qi(-3, '1/2'), (-3+1/2j))

        # int - float
        self.assertEqual(Qi(-3, 0.75), (-3+3/4j))

        # int - int
        self.assertEqual(Qi(-3, -3), (-3-3j))

        # int - Fr

In [10]:
from unittest import TestCase, TextTestRunner, defaultTestLoader
# from cayley_dickson_alg import Zi
# from random import seed
# from fractions import Fraction

class TestQi(TestCase):

    def setUp(self) -> None:
        seed(42)

    def test_constructor(self):
        #----------
        # RE - IM
        #----------
        # None - None
        self.assertEqual(Qi(), Qi(0, 0))

        # str - str
        self.assertEqual(Qi('1/2', '1/2'), (1/2+1/2j))

        # str - float
        self.assertEqual(Qi('1/2', 0.75), (1/2+3/4j))

        # str - int
        self.assertEqual(Qi('1/2', -3), (1/2-3j))

        # str - Fraction
        self.assertEqual(Qi('1/2', Fraction(3, 4)), (1/2+3/4j))

        # str - None
        self.assertEqual(Qi('1/2'), (1/2+0j))

        # float - str
        self.assertEqual(Qi(0.75, '1/2'), (3/4+1/2j))

        # float - float
        self.assertEqual(Qi(0.75, 0.75), (3/4+3/4j))

        # float - int
        self.assertEqual(Qi(0.75, -3), (3/4-3j))

        # float - Fraction
        self.assertEqual(Qi(0.75, Fraction(3, 4)), (3/4+3/4j))

        # float - None
        self.assertEqual(Qi(0.75), (3/4+0j))

        # int - str
        self.assertEqual(Qi(-3, '1/2'), (-3+1/2j))

        # int - float
        self.assertEqual(Qi(-3, 0.75), (-3+3/4j))

        # int - int
        self.assertEqual(Qi(-3, -3), (-3-3j))

        # int - Fraction
        self.assertEqual(Qi(-3, Fraction(3, 4)), (-3+3/4j))

        # int - None
        self.assertEqual(Qi(-3), (-3+0j))

        # Fraction - str
        self.assertEqual(Qi(Fraction(3, 4), '1/2'), (3/4+1/2j))

        # Fraction - float
        self.assertEqual(Qi(Fraction(3, 4), 0.75), (3/4+3/4j))

        # Fraction - int
        self.assertEqual(Qi(Fraction(3, 4), -3), (3/4-3j))

        # Fraction - Fraction
        self.assertEqual(Qi(Fraction(3, 4), Fraction(3, 4)), (3/4+3/4j))

        # Fraction - None
        self.assertEqual(Qi(Fraction(3, 4)), (3/4+0j))

        
        # complex - complex
        self.assertEqual(Qi((1-2j), (-3+5j)), Qi(Qi(1, -2), Qi(-3, 5)))
        self.assertEqual(Qi((1.5-2j), (-4.25+5j)), Qi(Qi(3/2, -2), Qi(-17/4, 5)))
        # complex - Qi
        # complex - Zi
        # complex - None
        self.assertEqual(Qi((1-2j)), Qi(1, -2))
        # Qi - complex
        self.assertEqual(Qi(Qi(1, -2), (-3+5j)), Qi(Qi(1, -2), Qi(-3, 5)))
        # Qi - Qi
        # Qi - Zi
        # Qi - None
        self.assertEqual(Qi(Qi(1, -2)), Qi(1, -2))
        # Zi - complex
        # Zi - Qi
        # Zi - Zi
        # Zi - None
        self.assertEqual(Qi(Zi(1, 2)), Qi(1, 2))

In [11]:
TextTestRunner(verbosity=2).run(defaultTestLoader.loadTestsFromTestCase(TestQi))

test_constructor (__main__.TestQi.test_constructor) ... FAIL

FAIL: test_constructor (__main__.TestQi.test_constructor)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/0y/ptqvq6x56tbdb9mz97r5tjk00000gn/T/ipykernel_28185/1748294088.py", line 19, in test_constructor
    self.assertEqual(Qi('1/2', '1/2'), (1/2+1/2j))
AssertionError: Qi(1/2, 1/2) != (0.5-0.5j)

----------------------------------------------------------------------
Ran 1 test in 0.006s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

In [ ]:
Qi('1/2', Fraction(3, 4))

In [ ]:
Qi(Zi(1, 2), Zi(3, 4))

In [ ]:
Qi('1/2', '1/2')